Import the necessary components. Some of them are new for this week, they will be discussed later in the notebook.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer, EarlyStoppingCallback, Seq2SeqTrainingArguments,Seq2SeqTrainer, DataCollatorForSeq2Seq
import torch
import time
import pandas as pd

import numpy as np
import re
from datasets import  load_from_disk,load_dataset


<a name='1.2'></a>
### 1.2 - Load Dataset and LLM


In [ ]:
# dataset = load_dataset("Ahmadsameh8/qalbPreprocessedAndMerged")

dataset = load_from_disk("qalb")
# dataset


In [ ]:
import gc
gc.collect()

24

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



In [ ]:
# from transformers import BitsAndBytesConfig


# nf4_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_compute_dtype=torch.bfloat16
# )
# quantization_config=nf4_config,

model_name = "model_new"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name,use_safetensors = True,device_map={"":0})
tokenizer = T5Tokenizer.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print_number_of_trainable_model_parameters(original_model)

'trainable model parameters: 367508736\nall model parameters: 367508736\npercentage of trainable model parameters: 100.00%'

In [ ]:
def preprocess_function(example,padding="max_length"):
    # add prefix to the input for t5
    inputs = example["incorrect"]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=512, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=example["correct"], max_length=512, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['correct', 'incorrect'])
print(f"Keys of tokenized dataset: {list(tokenized_datasets['train'].features)}")


Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Check the shapes of all three parts of the dataset:

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (18350, 3)
Validation: (2293, 3)
Test: (2295, 3)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18350
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2293
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2295
    })
})


<a name='2.2'></a>
### 2.2 - Fine-Tune the Model with the Preprocessed Dataset



In [ ]:
output_dir = "./textcorrection_model"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    learning_rate=5e-5, # Higher learning rate than full fine-tuning.
    num_train_epochs=4,
    # max_grad_norm=0.1,
    logging_steps=500,
    eval_steps=500,
    save_strategy= "epoch",
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    save_steps=1024,
    warmup_steps=512,
    load_best_model_at_end = True,
    fp16=True,
    save_total_limit=2,
    # report_to="tensorboard"
)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=4)

trainer = Seq2SeqTrainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],

    callbacks=[early_stopping_callback],
    data_collator=data_collator,


    # tokenizer=tokenizer,
)

In [ ]:
trainer.train()
model_path= "textcorrection"
trainer.model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

  1%|          | 500/73400 [02:37<6:24:33,  3.16it/s]

{'loss': 0.2258, 'grad_norm': 8.554731369018555, 'learning_rate': 4.8632812500000004e-05, 'epoch': 0.03}


  1%|▏         | 1000/73400 [05:14<6:20:33,  3.17it/s]

{'loss': 0.1818, 'grad_norm': 0.9594561457633972, 'learning_rate': 4.9667297771924045e-05, 'epoch': 0.05}


  2%|▏         | 1500/73400 [07:51<6:16:38,  3.18it/s]

...

                                                       
100%|██████████| 73400/73400 [6:32:11<00:00,  3.18it/s]

{'eval_loss': 0.48001521825790405, 'eval_runtime': 88.1994, 'eval_samples_per_second': 25.998, 'eval_steps_per_second': 25.998, 'epoch': 4.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
100%|██████████| 73400/73400 [6:32:16<00:00,  3.12it/s]


{'train_runtime': 23536.7797, 'train_samples_per_second': 3.119, 'train_steps_per_second': 3.119, 'train_loss': 0.13951776972258773, 'epoch': 4.0}


('textcorrection/tokenizer_config.json',
 'textcorrection/special_tokens_map.json',
 'textcorrection/spiece.model',
 'textcorrection/added_tokens.json')

In [ ]:
import gc
gc.collect()

835

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '{output_dir}'/runs

In [ ]:
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
tokenizer1 = T5Tokenizer.from_pretrained("textcorrection")
model1 = AutoModelForSeq2SeqLM.from_pretrained("textcorrection").to(device)

# ar_prompt="عاصمة ألمانيا هي <extra_id_0> "
input_ids = tokenizer1(dataset["test"]["incorrect"][2], return_tensors="pt").input_ids.to(device)
outputs = model1.generate(input_ids, max_length=512)
print("Tokenized input:", tokenizer1.tokenize(dataset["test"]["incorrect"][1]))
print("Decoded output:", tokenizer1.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenized input: ['▁علماء', '▁مصر', '▁ال', '▁ذ', 'ين', '▁ملء', 'وا', '▁الفضاء', 'حي', '▁ات', '▁لا', '▁يت', 'كل', '▁مون', '▁الا', '▁عن', '▁ن', 'واق', 'ض', '▁الوضوء', '▁و', '▁احكام', '▁دم', '▁الحي', 'ض', '▁و', '▁الن', '▁فاس', '▁ام', '▁ا', '▁دماء', '▁المسلمين', '▁الت', '▁ي', '▁يهد', 'رها', '▁الحك', '▁ام', '▁الظ', '▁لمه', '▁فلا', '▁احكام', '▁لها', 'في', '▁فقه', 'هم', '▁.', '▁وكيف', '▁نريد', 'هم', '▁ان', '▁ين', 'قد', 'وا', '▁من', '▁يسم', '▁ونه', '▁ولي', '▁امر', '▁و', '▁امير', '▁م', 'ء', 'منين', '▁و', '▁يحمد', 'ون', '▁الله', '▁علي', '▁الهواء', '▁عندما', '▁اجري', '▁عمليه', '▁جرا', 'حي', '▁ه', '▁ناج', 'حه', '▁في', '▁المانيا']
Decoded output: الكيل بمكيالين : الجزر للمصريين والعصا لاخواننا في افغانستان والعراق... امريكا اعلنت حربا صليبيه علي المسلمين علي لسان الجزار بوش ، وقتلت ، ودمرت ، وحاصرت ، وهي من ساندت الطاغيه لا مبارك ، لماذا ساندته ومولته ؟ عودوا الي المعتقلات ، والتصفيات ، والاختطافات فستجدون ان امريكا تخطط ولامبارك ينفذ هم العدو فاتلهم الله اني يءفكون.
